In [3]:
import pandas as pd
import numpy as np
from scipy import stats
from scipy.stats import shapiro
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from sklearn.ensemble import GradientBoostingRegressor, AdaBoostRegressor
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from sklearn.utils import resample
import math

# Especificar la ruta de carpeta y el prefijo del archivo para los "Received Goods"
folder_path_received_goods = 'W:/Database/Power Query Model Data/Received Goods/'
file_prefix_received_goods = 'tbl_received_goods_'

# Cargar los datos de "Received Goods" en un marco de datos de pandas
df_received_goods = pd.concat([pd.read_excel(folder_path_received_goods + file_prefix_received_goods + str(year) + '.xlsx') for year in range(2019, 2024)])

#ALTERNATIVA PARA SOLO UN ANO:
# Especificar el año que deseas cargar
#year = 2023
# Construir la ruta completa del archivo de datos
#file_path = folder_path_received_goods + file_prefix_received_goods + str(year) + '.xlsx'
# Cargar los datos del archivo en un marco de datos de pandas
#df_received_goods = pd.read_excel(file_path)

# Realizar tareas de limpieza y preprocesamiento de los datos
relevant_cols = ['Invoice No.', 'Order No.', 'Order Status', 'Transaction Month', 'Transaction Date', 'Order Date', 'Request Date', 'Product Category Code', 'Promise Date', 'Vendor Code', 'Vendor', 'Item Code', 'Receive Goods Qty.', 'Receive Goods Amt.', 'PO Qty.', 'PO Amt.', 'PO Price (Basic)', 'Order Type', 'Entry Date', 'Product Category Name3']
df_received_goods = df_received_goods[relevant_cols]

# Convertir las columnas de fechas a tipo datetime
date_columns = ['Transaction Date', 'Order Date', 'Request Date', 'Promise Date', 'Entry Date']
df_received_goods[date_columns] = df_received_goods[date_columns].apply(pd.to_datetime)

# Calcular el plazo de entrega restando la fecha de pedido a la fecha de transacción
# Calcular el plazo de entrega en días
df_received_goods['Plazo de Entrega (días)'] = (df_received_goods['Transaction Date'] - df_received_goods['Order Date']).dt.days

# Calcular el plazo de entrega en semanas
df_received_goods['Plazo de Entrega (semanas)'] = np.ceil(df_received_goods['Plazo de Entrega (días)'] / 7)

# Especificar el código de proveedor para explorar los datos
vendor_code = 70111

# Filtrar los datos para el proveedor especificado
vendor_data = df_received_goods[df_received_goods['Vendor Code'] == vendor_code]

In [4]:
# Calcular las estadísticas descriptivas del plazo de entrega para el proveedor
delivery_time_stats = vendor_data['Plazo de Entrega (semanas)'].describe()

# Calcular el rango intercuartil (IQR)
Q1 = delivery_time_stats['25%']
Q3 = delivery_time_stats['75%']
IQR = Q3 - Q1

# Calcular percentiles adicionales
percentiles = [10, 50, 90]
additional_percentiles = np.percentile(vendor_data['Plazo de Entrega (semanas)'], percentiles)

# Calcular estadísticas adicionales
additional_stats = stats.describe(vendor_data['Plazo de Entrega (semanas)'])

# Realizar la prueba de Shapiro-Wilk para verificar la normalidad de los plazos de entrega
statistic, p_value = shapiro(vendor_data['Plazo de Entrega (semanas)'])

# Dividir los datos en variables predictoras (X) y variable objetivo (y)
X = vendor_data[['Receive Goods Qty.', 'PO Qty.', 'PO Price (Basic)']]
y = vendor_data['Plazo de Entrega (semanas)']

# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Modelos a evaluar
models = {
    'Regresión Lineal': LinearRegression(),
    'Gradient Boosting': GradientBoostingRegressor(random_state=42),
    'AdaBoost': AdaBoostRegressor(random_state=42),
    'XGBoost': XGBRegressor(random_state=42),
    'LightGBM': LGBMRegressor(random_state=42)
}

# Evaluar y comparar los modelos
best_model = None
best_mse = float('inf')

for name, model in models.items():
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)

    mse = mean_squared_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    mae = mean_absolute_error(y_test, y_pred)

       # Actualizar el mejor modelo en función del MSE
    if mse < best_mse:
        best_model = name
        best_mse = mse


# Obtener el modelo más óptimo
best_model_name = best_model

# Obtener el objeto del modelo óptimo
best_model = models[best_model_name]

# Preparar los datos de entrada para el modelo
X = vendor_data[['Receive Goods Qty.', 'PO Qty.', 'PO Price (Basic)']]
y = vendor_data['Plazo de Entrega (semanas)']

# Entrenar el modelo más óptimo en todos los datos
best_model.fit(X, y)

# Realizar la predicción del plazo de entrega promedio esperado utilizando bootstrap
bootstrap_samples = 1000
bootstrap_preds = np.zeros(bootstrap_samples)
for i in range(bootstrap_samples):
    X_bootstrap, y_bootstrap = resample(X, y, random_state=i)
    bootstrap_preds[i] = best_model.predict(X_bootstrap).mean()

# Calcular percentiles del 2.5 y 97.5 para obtener el intervalo de confianza del 95%
confidence_interval = np.percentile(bootstrap_preds, [2.5, 97.5])

# Realizar la predicción basada en el modelo más óptimo
input_data = np.array([[vendor_data['Receive Goods Qty.'].mean(), vendor_data['PO Qty.'].mean(), vendor_data['PO Price (Basic)'].mean()]])
average_delivery_time = best_model.predict(input_data)[0]

#Imprimir las estadísticas descriptivas, el IQR, los percentiles adicionales, las estadísticas adicionales y el resultado de la prueba Shapiro-Wilk
print("*********************************")
print("Descriptive statistics:")
print(delivery_time_stats)
print("IQR:", IQR)
print("*********************************")
print("Additional percentiles:")
for percentile, value in zip(percentiles, additional_percentiles):
    print(f"Percentile {percentile}%: {value:.2f}")
print("*********************************")
print("Additional statistics:")
print(f"Variance: {additional_stats.variance:.2f}")
print(f"Coefficient of Variation: {additional_stats.variance / additional_stats.mean:.2f}")
print(f"Skewness: {additional_stats.skewness:.2f}")
print(f"Kurtosis: {additional_stats.kurtosis:.2f}")
print("*********************************")
print("Shapiro-Wilk test:")
print(f"Statistic: {statistic:.4f}")
print(f"Value p: {p_value:.4f}")
print("*********************************")
print("Optimal model:", best_model_name)
print("Expected average delivery time:", average_delivery_time)
print(f"95% confidence interval for prediction: {confidence_interval}")


descriptive_stats = delivery_time_stats
iqr = IQR

# Percentiles adicionales:
additional_percentiles_dict = dict(zip(percentiles, additional_percentiles))

# Estadísticas adicionales:
additional_stats_dict = {
    "Varianza": additional_stats.variance,
    "Coeficiente de Variación": additional_stats.variance / additional_stats.mean,
    "Asimetría (Skewness)": additional_stats.skewness,
    "Curtosis (Kurtosis)": additional_stats.kurtosis
}

# Prueba de Shapiro-Wilk:
shapiro_wilk_test = {
    "Estadístico": statistic,
    "Valor p": p_value
}

# Modelo óptimo:
optimal_model_name = best_model_name

# Plazo de entrega promedio esperado:
average_delivery_time_value = average_delivery_time

# Intervalo de confianza del 95% para la predicción:
confidence_interval_value = confidence_interval


*********************************
Descriptive statistics:
count    904.000000
mean       4.650442
std        2.623165
min        2.000000
25%        3.000000
50%        4.000000
75%        5.000000
max       27.000000
Name: Plazo de Entrega (semanas), dtype: float64
IQR: 2.0
*********************************
Additional percentiles:
Percentile 10%: 2.00
Percentile 50%: 4.00
Percentile 90%: 8.00
*********************************
Additional statistics:
Variance: 6.88
Coefficient of Variation: 1.48
Skewness: 2.55
Kurtosis: 10.18
*********************************
Shapiro-Wilk test:
Statistic: 0.7531
Value p: 0.0000
*********************************
Optimal model: Regresión Lineal
Expected average delivery time: 4.650442477876106
95% confidence interval for prediction: [4.60358675 4.70296418]


C:\Users\pablo.hacker\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(


In [5]:
def calculate_delivery_time(predicted_delivery, cv, skewness, percentile_90, IQR):
    # Umbrales definidos
    cv_threshold = 0.5
    skewness_threshold = (-1, 1)

    # Revisar si los datos se encuentran dentro de los umbrales
    if cv <= cv_threshold and skewness_threshold[0] <= skewness <= skewness_threshold[1]:
        print("Data are within the established thresholds. No additional buffer time is needed.")
        return predicted_delivery

    else:
        print("Data are not within the established thresholds. Additional buffer time is required.")

        # Cálculo del buffer time usando el percentil 90%
        buffer_time_percentile = percentile_90
        final_delivery_percentile = predicted_delivery + buffer_time_percentile

        # Cálculo del buffer time usando el IQR
        buffer_time_IQR = IQR
        final_delivery_IQR = predicted_delivery + buffer_time_IQR

        # Retornar el menor resultado de ambos
        if final_delivery_percentile <= final_delivery_IQR:
            print(f"The final lead time using the 90% percentile is: {final_delivery_percentile:.2f} weeks")
            return final_delivery_percentile
        else:
            print(f"The final lead time using the IQR is: {final_delivery_IQR:.2f} weeks")
            return final_delivery_IQR


# Obtener los valores necesarios de "part_stats"
predicted_delivery = average_delivery_time_value
cv = additional_stats_dict["Coeficiente de Variación"]
skewness = additional_stats_dict["Asimetría (Skewness)"]
percentile_90 = additional_percentiles_dict[90]
IQR = iqr

# Calcular el plazo de entrega final
delivery_time = calculate_delivery_time(predicted_delivery, cv, skewness, percentile_90, IQR)

# Mostrar el resultado redondeado hacia arriba
rounded_delivery_time = math.ceil(delivery_time)
print(f"The final lead time for customer service is: {rounded_delivery_time} weeks")


Data are not within the established thresholds. Additional buffer time is required.
The final lead time using the IQR is: 6.65 weeks
The final lead time for customer service is: 7 weeks


In [4]:
import webbrowser

# Pregunta al usuario si quiere abrir el link
answer = input("Do you want to open the statistical report? Answer yes or no: ")

# Comprueba si la respuesta es "sí"
if answer.lower() == "yes":
    # URL del reporte en Google Drive
    url = "https://docs.google.com/document/d/191lF4VUEi_qt_8ZdQxNVBIeS1Deic-V48PWEYAU3DRE/edit?usp=sharing"  # Asegúrate de reemplazar esto con el URL de tu reporte

    # Abre el URL en el navegador web
    webbrowser.open(url)
